In [2]:
import pandas as pd
import numpy as np
import random
import re
from tqdm import tqdm
import xml.etree.ElementTree as ET

In [3]:
df_abcd = pd.read_parquet("../data/ABCD_tripfiles.parquet", engine="pyarrow")
df_abcd.head()

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details
0,33902381,2024-05-01 09:14:42,AB,2359,1,BLR,service-acco,ASMMsgProcessor,"2024-05-01 09:14:42,420 INFO [3f326d134d9de45...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
1,33910427,2024-05-01 10:28:12,AB,2104,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,745 INFO [de24be75947007f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
2,33910380,2024-05-01 10:28:11,AB,2109,2,BLR,service-acco,ASMMsgProcessor,"2024-05-01 10:28:11,301 INFO [df3a1cc7d574161...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
3,33909790,2024-05-01 10:28:12,AB,2110,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 10:28:12,189 INFO [9e66dfb1ffe9048...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."
4,33902706,2024-05-01 09:14:41,AB,2127,2,BOM,service-acco,ASMMsgProcessor,"2024-05-01 09:14:41,884 INFO [829566f527b588f...","<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<ns2..."


In [4]:
df_abcd.action_name.value_counts().to_dict()

{'CalculateWeightAndTrimAction': 231990,
 'StorePaxDataAction': 150871,
 'CheckinMsgProcessor': 133782,
 'UpdateLoadTableAction': 23554,
 'ASMMsgProcessor': 22438,
 'AssignLCCAction': 18162,
 'UpdateFlightAction': 9468,
 'PAXBOOKINGINMsgProcessor': 8878,
 'ChangeFlightLegStateAction': 8836,
 'StoreRegistrationAndConfigurationAc': 8036,
 'UpdateSupplementaryInfoAction': 7936,
 'UpdateCrewDataAction': 7173,
 'TransferCheckinDataAction': 6586,
 'CreateLoadingInstructionAction': 6408,
 'CrewMsgProcessor': 5444,
 'EstimateStorePaxDataAction': 5069,
 'CreateLoadsheetAction': 5055,
 'CreateZFWMessageAction': 5005,
 'RampFinalAction': 3804,
 'UpdateFuelDataAction': 3583,
 'CreatePostDepartureMessagesAction': 3406,
 'AssignUnassignViewAction': 3384,
 'ResetLoadingListRecordsAction': 2955,
 'SendPostDepartureMessagesAction': 2728,
 'FlightPlanFiguresInMsgProcessor': 2346,
 'AssignLoadplanAction': 2284,
 'TransferCargoAction': 2086,
 'SetActualBagWeightIndicatorAction': 1978,
 'CloseLegAction': 1

## **Functions**

In [5]:
def write_to_file(text: str, fileneame: str):
    with open(fileneame, "w") as file:
        file.write(text)

def add_to_df(df, extracted_data, idx):
    for key, value in extracted_data.items():
            # add columns from entry_details keys if not exist
            if key not in df.columns:
                df[str(key)] = np.nan
            # write data in entry_details key columns
            df.at[idx, key] = value


## **CreateZFWMessageAction**

In [6]:
CreateZFWMessageAction_df = df_abcd[(df_abcd["action_name"] == "CreateZFWMessageAction")]
CreateZFWMessageAction_df.head()

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details
423238,33767641,2024-04-30 04:04:17,AB,2519,30,COK,human,CreateZFWMessageAction,"2024-04-30 04:04:17,021 INFO [4156a3c5a78e995...",EmptyDTO - no input expected
423239,33767642,2024-04-30 04:04:17,AB,2519,30,COK,human,CreateZFWMessageAction,"2024-04-30 04:04:17,022 INFO [4156a3c5a78e995...",Receiver queue : loadsystem/master/queue/...
423240,33767624,2024-04-30 04:04:58,AB,2627,30,GOX,human,CreateZFWMessageAction,"2024-04-30 04:04:58,954 INFO [fd01af5585fd5e1...",EmptyDTO - no input expected
423241,33767625,2024-04-30 04:04:58,AB,2627,30,GOX,human,CreateZFWMessageAction,"2024-04-30 04:04:58,954 INFO [fd01af5585fd5e1...",Receiver queue : loadsystem/master/queue/...
423242,33767576,2024-04-30 04:05:53,AB,2498,30,BOM,human,CreateZFWMessageAction,"2024-04-30 04:05:53,897 INFO [c5d021fdea18fd7...",EmptyDTO - no input expected


In [7]:
random_index = random.randint(0, len(CreateZFWMessageAction_df) - 1)
random_sample = str(CreateZFWMessageAction_df.iloc[random_index].entry_details)
pd.set_option('display.max_colwidth', None)
random_sample
write_to_file(random_sample, "message.txt")
print(random_sample)

EmptyDTO - no input expected


In [8]:
def extract_data_from_xml(xml_string):
    # Use regular expressions to find key-value pairs
    keys = ['airline', 'arrivalStation', 'departureStation', 'flightDateLocal', 'revisionNumber',
            'dryOperatingWeight', 'actualZFW', 'cargoWeight', 'baggageWeight', 'paxWeight', 'basicWeight']

    data_dict = {}
    for key in keys:
        match = re.search(r'<{}>(.*?)</{}>'.format(key, key), xml_string)
        if match:
            data_dict[key] = match.group(1)
        else:
            data_dict[key] = None
    return data_dict

extracted_data = extract_data_from_xml(random_sample)
print("Extracted data:")
print(extracted_data)     


Extracted data:
{'airline': None, 'arrivalStation': None, 'departureStation': None, 'flightDateLocal': None, 'revisionNumber': None, 'dryOperatingWeight': None, 'actualZFW': None, 'cargoWeight': None, 'baggageWeight': None, 'paxWeight': None, 'basicWeight': None}


In [9]:

# create columns and store information from entry_deatails in them
i = 0
for idx, row in tqdm(CreateZFWMessageAction_df.iterrows(), total=CreateZFWMessageAction_df.shape[0], desc="Processing rows"):
    details = row["entry_details"]

    try:
        # skip system row because of lacking information
        if "Receiver queue" in details:
            extracted_data = extract_data_from_xml(details)
            add_to_df(CreateZFWMessageAction_df, extracted_data, idx)
    except Exception as e:
        print(e)
        write_to_file(random_sample, "message.txt")
        break
    else:
        pass
print(i)

Processing rows:   0%|          | 0/5005 [00:00<?, ?it/s]C:\Users\janmu\AppData\Local\Temp\ipykernel_20052\1624572607.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(key)] = np.nan
C:\Users\janmu\AppData\Local\Temp\ipykernel_20052\1624572607.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'AB' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, key] = value
C:\Users\janmu\AppData\Local\Temp\ipykernel_20052\1624572607.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

0


In [10]:
CreateZFWMessageAction_df.columns

Index(['id', 'creation_time', 'airline_code', 'flight_number', 'flight_date',
       'departure_airport', 'user_name', 'action_name', 'header_line',
       'entry_details', 'airline', 'arrivalStation', 'departureStation',
       'flightDateLocal', 'revisionNumber', 'dryOperatingWeight', 'actualZFW',
       'cargoWeight', 'baggageWeight', 'paxWeight', 'basicWeight'],
      dtype='object')

In [11]:
CreateZFWMessageAction_df.head(1000)

,id,creation_time,airline_code,flight_number,flight_date,departure_airport,user_name,action_name,header_line,entry_details,...,arrivalStation,departureStation,flightDateLocal,revisionNumber,dryOperatingWeight,actualZFW,cargoWeight,baggageWeight,paxWeight,basicWeight
423238,33767641,2024-04-30 04:04:17,AB,2519,30,COK,human,CreateZFWMessageAction,"2024-04-30 04:04:17,021 INFO [4156a3c5a78e995a] [AB2519 /30 COK | human | CreateZFWMessageAction | | default task-22] Received:",EmptyDTO - no input expected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423239,33767642,2024-04-30 04:04:17,AB,2519,30,COK,human,CreateZFWMessageAction,"2024-04-30 04:04:17,022 INFO [4156a3c5a78e995a] [AB2519 /30 COK | human | CreateZFWMessageAction | | default task-22] Sent:","Receiver queue : loadsystem/master/queue/LOADC.SEND.LOADINFO\r\nDocument :\r\n<?xml version=""1.0"" encoding=""UTF-8"" standalone=""yes""?>\r\n<ns14:ZFWFigures xmlns:ns2=""http://www.iata.org/IATA/2007/00"" xmlns:ns4=""http://www.lsb.de/iocc/passengerBookingFigures"" xmlns:ns3=""http://www.lsb.de/iocc/FlightPlanFigures"" xmlns:ns6=""http://www.lsb.de/iocc/IncomingNTMMessage"" xmlns:ns20=""http://www.onesystem.com/iocc/nll/AcarsLsAcknowledge"" xmlns:ns5=""http://www.lsb.de/iocc/AcarsInXml"" xmlns:ns8=""http://www.onesystem.com/iocc/nll/CateringLsAcknowledge"" xmlns:ns7=""http://www.lsb.de/iocc/PassengerCheckIn"" xmlns:ns13=""http://www.lsb.de/iocc/LoadSheetInXml"" xmlns:ns9=""http://www.lsb.de/iocc/CkiStatusInformation"" xmlns:ns12=""http://www.lsb.de/iocc/Email"" xmlns:ns11=""http://www.lsb.de/iocc/BaggageUnitMessage"" xmlns:ns10=""http://www.lsb.de/iocc/LoadInstructionXML"" xmlns:ns17=""http://www.lsb.de/iocc/CargoForecastMessage"" xmlns:ns16=""http://www.lsb.de/iocc/CrewFiguresMessageINT"" xmlns:ns15=""http://www.lsb.de/iocc/AircraftPositionFigures"" xmlns:ns14=""http://www.lsb.de/iocc/zeroFuelWeightMessage"" xmlns:ns19=""http://www.lsb.de/iocc/RampFuelFigures"" xmlns:ns18=""http://www.lsb.de/iocc/RequestMessageFigures"">\r\n <loadsystemHeader>\r\n <airline>AB</airline>\r\n <flightNumber>2519</flightNumber>\r\n <legNumber>1</legNumber>\r\n <arrivalStation>BOM</arrivalStation>\r\n <departureStation>COK</departureStation>\r\n <flightDateLocal>2024-04-30</flightDateLocal>\r\n <flightDateUTC>2024-04-30</flightDateUTC>\r\n <std>300640</std>\r\n </loadsystemHeader>\r\n <timestamp>2024-04-30T04:04:16Z</timestamp>\r\n <planningStatus>2</planningStatus>\r\n <revisionNumber>2</revisionNumber>\r\n <weightUnit>K</weightUnit>\r\n <dryOperatingWeight>44584</dryOperatingWeight>\r\n <estimTotalTrafficLoad>14416</estimTotalTrafficLoad>\r\n <baggageIndicator>true</baggageIndicator>\r\n <mailIndicator>true</mailIndicator>\r\n <transitLoadIndicator>false</transitLoadIndicator>\r\n <tailTankIndicator>false</tailTankIndicator>\r\n <actualZFW>59000</actualZFW>\r\n <numberPassenger>160</numberPassenger>\r\n <cargoWeight>700</cargoWeight>\r\n <mailWeight>66</mailWeight>\r\n <baggageWeight>1800</baggageWeight>\r\n <paxWeight>11850</paxWeight>\r\n <basicWeight>44170</basicWeight>\r\n <basicIndex>+39.79</basicIndex>\r\n <userID>human</userID>\r\n <systemZFWMessageIndicator>false</systemZFWMessageIndicator>\r\n <manualCalculationIndicator>true</manualCalculationIndicator>\r\n <ns14:ZFWPaxPerClass>\r\n <classCode>Y</classCode>\r\n <classCapacity>189</classCapacity>\r\n <numberOfPax>0</numberOfPax>\r\n </ns14:ZFWPaxPerClass>\r\n</ns14:ZFWFigures>",...,BOM,COK,2024-04-30,2,44584,59000,700,1800,11850,44170
423240,33767624,2024-04-30 04:04:58,AB,2627,30,GOX,human,CreateZFWMessageAction,"2024-04-30 04:04:58,954 INFO [fd01af5585fd5e1a] [AB2627 /30 GOX | human | CreateZFWMessageAction | | default task-57] Received:",EmptyDTO - no input expected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423241,33767625,2024-04-30 04:04:58,AB,2627,30,GOX,human,CreateZFWMessageAction,"2024-04-30 04:04:58,954 INFO [fd01af5585fd5e1a] [AB2627 /30 GOX | human | CreateZFWMessageAction | | default task-57] Sent:","Receiver queue : loadsystem/